In [1]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
import numpy as np

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [3]:
negative_words = {
    'no',
    'not',
    'none',
    'neither',
    'never',
    'nobody',
    'nothing',
    'nowhere',
    "doesn't",
    "isn't",
    "wasn't",
    "shouldn't",
    "won't",
    "can't",
    "couldn't",
    "don't",
    "doesn't",
    "haven't",
    "hasn't",
    "hadn't",
    "aren't",
    "weren't",
    "wouldn't",
    "daren't",
    "needn't",
    "didn't",
    'without',
    'against',
    'negative',
    'deny',
    'reject',
    'refuse',
    'decline',
    'unhappy',
    'sad',
    'miserable',
    'hopeless',
    'worthless',
    'useless',
    'futile',
    'disagree',
    'oppose',
    'contrary',
    'contradict',
    'disapprove',
    'dissatisfied',
    'objection',
    'unsatisfactory',
    'unpleasant',
    'regret',
    'resent',
    'lament',
    'mourn',
    'grieve',
    'bemoan',
    'despise',
    'loathe',
    'detract',
    'abhor',
    'dread',
    'fear',
    'worry',
    'anxiety',
    'sorrow',
    'gloom',
    'melancholy',
    'dismay',
    'disheartened',
    'despair',
    'dislike',
    'aversion',
    'antipathy',
    'hate',
    'disdain'
}

In [4]:
stop_word_without_negative = [word for word in stop_words if word not in negative_words]
stop_word_without_negative

['to',
 'a',
 'themselves',
 'was',
 'just',
 'have',
 'than',
 'below',
 'over',
 'yourself',
 'in',
 'or',
 'as',
 'myself',
 'hadn',
 'does',
 'so',
 'mustn',
 'has',
 'now',
 'himself',
 'doesn',
 "that'll",
 't',
 'into',
 'them',
 'few',
 'by',
 'where',
 'any',
 'me',
 'your',
 'these',
 'll',
 'same',
 'don',
 'ours',
 'under',
 'from',
 've',
 "it's",
 'it',
 'while',
 'shan',
 'haven',
 'yours',
 'why',
 'do',
 'will',
 'but',
 'about',
 'i',
 'be',
 'with',
 'is',
 'the',
 'my',
 'they',
 'aren',
 'very',
 'both',
 "you'd",
 "shan't",
 'other',
 'weren',
 'during',
 'there',
 "you'll",
 'couldn',
 'herself',
 'what',
 'itself',
 'his',
 'did',
 'which',
 'again',
 'can',
 'between',
 'her',
 'should',
 'all',
 "mustn't",
 'having',
 'been',
 'isn',
 'doing',
 'of',
 'o',
 'our',
 'before',
 'when',
 'wouldn',
 's',
 'above',
 'whom',
 'that',
 'being',
 'an',
 'y',
 "mightn't",
 'theirs',
 'here',
 'those',
 'up',
 're',
 'wasn',
 'through',
 'this',
 'on',
 'ourselves',
 't

In [5]:
def pre_processing(line):
    line = line.lower()
    line = re.sub(r'[^a-zA-Z ]', '', line).strip()
    tokens = word_tokenize(line)
    #filtered_tokens = [word for word in tokens if word.lower() not in stop_word_without_negative]
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    lemmatized_text = " ".join(lemmatized_tokens)
    return lemmatized_text

In [6]:
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras import optimizers

from tensorflow import keras
from keras.layers import Flatten, Dense, Conv2D, MaxPooling2D, Input, Dropout, LSTM, Embedding
from keras.models import Model, Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

In [7]:
f = open("D:\\sentiment.txt", "r")
lines_with_label = f.readlines()

In [8]:
lines = []
labels = []
for line in lines_with_label:
    tmp = line.split('\t')
    lines.append(tmp[0].strip())
    labels.append(tmp[1].strip())

In [9]:
lines_preprocessed = []

for line in lines:
    lines_preprocessed.append(pre_processing(line))

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

In [11]:
tfidf_matrix = vectorizer.fit_transform(lines_preprocessed)

In [12]:
lines_vectorized = tfidf_matrix.toarray()

In [13]:
labels = np.array(labels)

In [14]:
labels = labels.astype(int)

In [15]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(lines_vectorized, labels, test_size=0.2)

In [16]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)

In [17]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
callbacks = [
    ModelCheckpoint('best_model.h5', verbose = 1, save_best_only = True)
    #EarlyStopping(monitor = "val_loss", patience = 10, restore_best_weights=False)
]

In [18]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Flatten())
#model.add(LSTM(units=8))
#model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(Dropout(0.25))
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dense(8, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [19]:
model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=128,
    validation_data=(X_val, y_val),
    callbacks=[callbacks]
)

Epoch 1/100
1/7 [===>..........................] - ETA: 3s - loss: 0.6951 - accuracy: 0.4609
Epoch 1: val_loss improved from inf to 0.68831, saving model to best_model.h5
7/7 [==============================] - 2s 300ms/step - loss: 0.6926 - accuracy: 0.5125 - val_loss: 0.6883 - val_accuracy: 0.5900
Epoch 2/100
1/7 [===>..........................] - ETA: 0s - loss: 0.6779 - accuracy: 0.7344
Epoch 2: val_loss improved from 0.68831 to 0.67502, saving model to best_model.h5
7/7 [==============================] - 0s 54ms/step - loss: 0.6716 - accuracy: 0.7425 - val_loss: 0.6750 - val_accuracy: 0.6900
Epoch 3/100
1/7 [===>..........................] - ETA: 0s - loss: 0.6503 - accuracy: 0.8125
Epoch 3: val_loss improved from 0.67502 to 0.65759, saving model to best_model.h5
7/7 [==============================] - 0s 30ms/step - loss: 0.6429 - accuracy: 0.8413 - val_loss: 0.6576 - val_accuracy: 0.7700
Epoch 4/100
1/7 [===>..........................] - ETA: 0s - loss: 0.6161 - accuracy: 0.8984
E

Epoch 30/100
1/7 [===>..........................] - ETA: 0s - loss: 0.0386 - accuracy: 0.9922
Epoch 30: val_loss did not improve from 0.41864
7/7 [==============================] - 0s 13ms/step - loss: 0.0542 - accuracy: 0.9775 - val_loss: 0.5118 - val_accuracy: 0.8200
Epoch 31/100
1/7 [===>..........................] - ETA: 0s - loss: 0.0634 - accuracy: 0.9844
Epoch 31: val_loss did not improve from 0.41864
7/7 [==============================] - 0s 12ms/step - loss: 0.0610 - accuracy: 0.9762 - val_loss: 0.5199 - val_accuracy: 0.8150
Epoch 32/100
1/7 [===>..........................] - ETA: 0s - loss: 0.0320 - accuracy: 0.9922
Epoch 32: val_loss did not improve from 0.41864
7/7 [==============================] - 0s 12ms/step - loss: 0.0533 - accuracy: 0.9800 - val_loss: 0.5220 - val_accuracy: 0.8200
Epoch 33/100
1/7 [===>..........................] - ETA: 0s - loss: 0.0527 - accuracy: 0.9844
Epoch 33: val_loss did not improve from 0.41864
7/7 [==============================] - 0s 13ms/s

1/7 [===>..........................] - ETA: 0s - loss: 0.0820 - accuracy: 0.9688
Epoch 60: val_loss did not improve from 0.41864
7/7 [==============================] - 0s 12ms/step - loss: 0.0507 - accuracy: 0.9787 - val_loss: 0.6629 - val_accuracy: 0.8050
Epoch 61/100
1/7 [===>..........................] - ETA: 0s - loss: 0.0290 - accuracy: 1.0000
Epoch 61: val_loss did not improve from 0.41864
7/7 [==============================] - 0s 13ms/step - loss: 0.0482 - accuracy: 0.9837 - val_loss: 0.6637 - val_accuracy: 0.8100
Epoch 62/100
1/7 [===>..........................] - ETA: 0s - loss: 0.0411 - accuracy: 0.9766
Epoch 62: val_loss did not improve from 0.41864
7/7 [==============================] - 0s 12ms/step - loss: 0.0546 - accuracy: 0.9737 - val_loss: 0.6673 - val_accuracy: 0.8100
Epoch 63/100
1/7 [===>..........................] - ETA: 0s - loss: 0.0623 - accuracy: 0.9844
Epoch 63: val_loss did not improve from 0.41864
7/7 [==============================] - 0s 12ms/step - loss: 0

7/7 [==============================] - 0s 16ms/step - loss: 0.0431 - accuracy: 0.9762 - val_loss: 0.7168 - val_accuracy: 0.7950
Epoch 91/100
1/7 [===>..........................] - ETA: 0s - loss: 0.0482 - accuracy: 0.9844
Epoch 91: val_loss did not improve from 0.41864
7/7 [==============================] - 0s 13ms/step - loss: 0.0462 - accuracy: 0.9825 - val_loss: 0.7187 - val_accuracy: 0.7900
Epoch 92/100
1/7 [===>..........................] - ETA: 0s - loss: 0.0473 - accuracy: 0.9766
Epoch 92: val_loss did not improve from 0.41864
7/7 [==============================] - 0s 13ms/step - loss: 0.0407 - accuracy: 0.9825 - val_loss: 0.7122 - val_accuracy: 0.8050
Epoch 93/100
1/7 [===>..........................] - ETA: 0s - loss: 0.0476 - accuracy: 0.9844
Epoch 93: val_loss did not improve from 0.41864
7/7 [==============================] - 0s 12ms/step - loss: 0.0467 - accuracy: 0.9787 - val_loss: 0.7154 - val_accuracy: 0.8050
Epoch 94/100
1/7 [===>..........................] - ETA: 0s - 

In [20]:
from tensorflow.keras.models import load_model

In [21]:
best_model = load_model('best_model.h5')

loss, accuracy = best_model.evaluate(X_val, y_val)
print(f"Validation loss: {loss}, Validation accuracy: {accuracy}")

7/7 [==============================] - 0s 2ms/step - loss: 0.4186 - accuracy: 0.8400
Validation loss: 0.41864052414894104, Validation accuracy: 0.8399999737739563
